In [ ]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb

In [66]:

def Stacking_Regression(X_train, X_test, y_train, y_test, filtered_data, param_dict):
    ml_type = 'SL'

    # Define base learners
    rf_regressor = RandomForestRegressor(n_estimators=param_dict[0]['n_estimators'], 
                                         max_depth=param_dict[0]['max_depth'],
                                         min_samples_leaf=param_dict[0]['min_samples_leaf'],
                                         min_samples_split=param_dict[0]['min_samples_split'], 
                                         random_state=0)

    bagging_regressor = BaggingRegressor(estimator=DecisionTreeRegressor(), 
                                         n_estimators=param_dict[1]['n_estimators'],
                                         max_samples=param_dict[1]['max_samples'], 
                                         max_features=param_dict[1]['max_features'],
                                         random_state=42)

    lgbm_regressor = LGBMRegressor(n_estimators=param_dict[2]['n_estimators'], 
                                   max_depth=param_dict[2]['max_depth'],
                                   learning_rate=param_dict[2]['learning_rate'], 
                                   num_leaves=param_dict[2]['num_leaves'],
                                   min_child_samples=param_dict[2]['min_child_samples'], 
                                   random_state=1)

    nn_regressor = MLPRegressor(hidden_layer_sizes=(64, 64), activation='relu', random_state=1, max_iter=500)
    # Define the final estimator as Ridge
    final_estimator = Ridge(alpha=1.0, random_state=42)  # You can tune the alpha parameter

    # Define the stacking regressor with Ridge as the final estimator
    model = StackingRegressor(
        estimators=[
            ('rf', rf_regressor),
            ('bagging', bagging_regressor),
            ('lgbm', lgbm_regressor),
            ('nn', nn_regressor)],
        final_estimator=final_estimator
    )

    # Train the model and make predictions as before
    model.fit(X_train, y_train)
    predict_train = model.predict(X_train)
    predict_test = model.predict(X_test)

    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    # Concatenate the DataFrames
    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()
    print(combined_df)

    r2_score_train = round(metrics.r2_score(y_train, predict_train), 2) 
    r2_score_test = round(metrics.r2_score(y_test, predict_test), 2)
    print('r2 train', r2_score_train)    
    print('r2 test', r2_score_test) 
    
    if 'dCn' in filtered_data:
        unit = 'dCn'
    else:
        unit = 'pm_conc'
    
    # Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit,ml_type)
    # qq_plot(test_df, v, unit,ml_type)
    return r2_score_test

In [ ]:
# def Stacking_Regression(X_train, X_test, y_train, y_test, filtered_data, param_dict):
#     ml_type = 'SL'
    
#     #     Define base learners
#     rf_regressor = RandomForestRegressor(n_estimators=param_dict[0]['n_estimators'], 
#                                          max_depth=param_dict[0]['max_depth'],
#                                          min_samples_leaf=param_dict[0]['min_samples_leaf'],
#                                          min_samples_split=param_dict[0]['min_samples_split'], 
#                                          random_state=0)

#     bagging_regressor = BaggingRegressor(estimator=DecisionTreeRegressor(), 
#                                          n_estimators=param_dict[1]['n_estimators'],
#                                          max_samples=param_dict[1]['max_samples'], 
#                                          max_features=param_dict[1]['max_features'],
#                                          random_state=42)

#     lgbm_regressor = LGBMRegressor(n_estimators=param_dict[2]['n_estimators'], 
#                                    max_depth=param_dict[2]['max_depth'],
#                                    learning_rate=param_dict[2]['learning_rate'], 
#                                    num_leaves=param_dict[2]['num_leaves'],
#                                    min_child_samples=param_dict[2]['min_child_samples'], 
#                                    random_state=1)

#     nn_regressor = MLPRegressor(hidden_layer_sizes=(64, 64), activation='relu', random_state=1, max_iter=500)

#     def ridge_cv(alpha):
#         ridge_regressor = Ridge(alpha=alpha, random_state=42)

#         # Define the stacking regressor with Ridge as the final estimator
#         model = StackingRegressor(
#             estimators=[
#                 ('rf', rf_regressor),
#                 ('bagging', bagging_regressor),
#                 ('lgbm', lgbm_regressor),
#                 ('nn', nn_regressor)
#             ],
#             final_estimator=ridge_regressor
#         )

#         # Fit the model and return the cross-validation score
#         model.fit(X_train, y_train)
#         predict_test = model.predict(X_test)
#         return r2_score(y_test, predict_test)

#     # Bayesian Optimization for Ridge alpha
#     pbounds = {'alpha': (0.01, 100)}

#     optimizer = BayesianOptimization(
#         f=ridge_cv,
#         pbounds=pbounds,
#         random_state=1
#     )
#     optimizer.maximize(init_points=10, n_iter=25)

#     # Optimal alpha
#     optimal_alpha = round(optimizer.max['params']['alpha'],4)
#     print("Optimal Alpha for Ridge:", optimal_alpha)

#     # Re-train the model with the optimal alpha
#     final_ridge_regressor = Ridge(alpha=optimal_alpha, random_state=42)
#     final_model = StackingRegressor(
#         estimators=[
#             ('rf', rf_regressor),
#             ('bagging', bagging_regressor),
#             ('lgbm', lgbm_regressor),
#             ('nn', nn_regressor)
#         ],
#         final_estimator=final_ridge_regressor
#     )
#     final_model.fit(X_train, y_train)

#     # Predictions and Evaluation
#     predict_train = final_model.predict(X_train)
#     predict_test = final_model.predict(X_test)

#     train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
#     test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

#     combined_df = pd.concat([train_df, test_df])
#     combined_df = combined_df.sort_index()

#     r2_score_train = round(r2_score(y_train, predict_train), 2)
#     r2_score_test = round(r2_score(y_test, predict_test), 2)

#     print('R2 Score Train:', r2_score_train)
#     print('R2 Score Test:', r2_score_test)
    
    
#     if 'dCn' in filtered_data:
#         unit = 'dCn'
#     else:
#         unit = 'pm_conc'
    
#     Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit,ml_type)
#     qq_plot(test_df, v, unit,ml_type)
#     return r2_score_test